In [ ]:
from scipy.stats import linregress
import numpy as np
import netCDF4 as nc
import pandas as pd
from statsmodels.tsa.seasonal import STL

# Global Ocean Colour (Copernicus-GlobColour), Bio-Geo-Chemical, L4 (monthly and interpolated) from Satellite Observations (Near Real Time)
file_id = nc.Dataset('/home/jamie/projects/climate/data/chl/chl_1998_2023_l4_month_multi_4k.nc')
ras = file_id.variables["CHL"][:]
time = file_id.variables["time"][:]
lat = file_id.variables["latitude"][:]
lon = file_id.variables["longitude"][:]
file_id.close()

# Initialize arrays to hold the slopes and p-values for the seascli component
seascli_slopes = np.zeros((ras.shape[1], ras.shape[2]))
seascli_p_values = np.ones((ras.shape[1], ras.shape[2]))

In [ ]:
ts = ras[:, i, j]
  
# Skip cells with all NaN values
if np.all(np.isnan(ts)):
    seascli_slopes[i, j] = np.nan
    seascli_p_values[i, j] = np.nan
    continue
        
# Convert to pandas Series
pix = pd.Series(ts, index=pd.date_range("1-1-1998", periods=len(ts), freq="M"), name="chl")
# Apply STL decomposition
stl = STL(pix, seasonal=13, robust=True)
fit = stl.fit()
        
# Compute the seascli component: trend + residual
trend = fit.trend #.dropna()  # Extract trend component
residual = fit.resid #.dropna()  # Extract residual (anomaly) component

In [ ]:

# Perform linear regression for the seascli component of each (lat, lon) cell
for i in range(ras.shape[1]):  # Loop over rows (latitude)
    for j in range(ras.shape[2]):  # Loop over columns (longitude)
        # Extract time series for the (i, j) cell
        ts = ras[:, i, j]
        
        # Skip cells with all NaN values
        if np.all(np.isnan(ts)):
            seascli_slopes[i, j] = np.nan
            seascli_p_values[i, j] = np.nan
            continue
        
        # Convert to pandas Series
        pix = pd.Series(ts, index=pd.date_range("1-1-1998", periods=len(ts), freq="M"), name="chl")

        # Apply STL decomposition
        stl = STL(pix, seasonal=13, robust=True)
        fit = stl.fit()
        
        # Compute the seascli component: trend + residual
        trend = fit.trend #.dropna()  # Extract trend component
        residual = fit.resid #.dropna()  # Extract residual (anomaly) component
        
        # Ensure both components align in time
        # common_index = trend.index.intersection(residual.index)
        # seascli = trend.loc[common_index] + residual.loc[common_index]
        seascli = trend + residual
        
        # Perform linear regression on the seascli component
        # x = (seascli.index - seascli.index[0]).days  # Time in days as the independent variable
        x = seascli.index  # Time in days as the independent variable
        y = seascli.values
        
        # Compute slope and p-value using linregress
        slope, _, _, p_value, _ = linregress(x, y)
        seascli_slopes[i, j] = slope  # Store the slope
        seascli_p_values[i, j] = p_value  # Store the p-value
